In [1]:
import numpy as np
import torch
import dxtb
from dxtb.typing import DD
from dxtb.config import ConfigCache
from dxtb import OutputHandler

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}

# LiH
numbers = torch.tensor([3, 1], device=dd["device"])
positions = torch.tensor([[0.0, 0.0, 0.0], [0.0, 0.0, 1.5]], **dd) # ** to use dd as kwargs 

# numbers = torch.tensor([6, 6, 7, 7, 1, 1, 1, 1, 1, 1, 8, 8,], device=dd["device"])
# positions = torch.tensor([
#                 [-3.81469488143921, +0.09993441402912, 0.00000000000000],
#                 [+3.81469488143921, -0.09993441402912, 0.00000000000000],
#                 [-2.66030049324036, -2.15898251533508, 0.00000000000000],
#                 [+2.66030049324036, +2.15898251533508, 0.00000000000000],
#                 [-0.73178529739380, -2.28237795829773, 0.00000000000000],
#                 [-5.89039325714111, -0.02589114569128, 0.00000000000000],
#                 [-3.71254944801331, -3.73605775833130, 0.00000000000000],
#                 [+3.71254944801331, +3.73605775833130, 0.00000000000000],
#                 [+0.73178529739380, +2.28237795829773, 0.00000000000000],
#                 [+5.89039325714111, +0.02589114569128, 0.00000000000000],
#                 [-2.74426102638245, +2.16115570068359, 0.00000000000000],
#                 [+2.74426102638245, -2.16115570068359, 0.00000000000000],
#                 ], **dd) # ** to use dd as kwargs

pos = positions.clone().requires_grad_(True)

# instantiate a calculator
cache_config = ConfigCache(enabled=True, coefficients=True, mo_energies=True, density=True)
cache_config = ConfigCache(enabled=True, coefficients=True, mo_energies=True, density=True, overlap=True)
calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, CACHE_STORE_DENSITY=True, **dd)
calc.opts.cache = cache_config

OutputHandler.verbosity = 0

def get_jacobian(matrix, pos):
    matrix_jac = torch.zeros(matrix.shape + pos.shape, dtype=matrix.dtype)
    for i, j in np.ndindex(matrix.shape):
        matrix_jac[i, j, :, :] = torch.autograd.grad(matrix[i, j], pos, create_graph=True, retain_graph=True)[0]
    return matrix_jac

In [2]:
from tqdm import tqdm  

energy = calc.get_energy(pos)
overlap = calc.integrals.build_overlap(pos)
hcore = calc.integrals.build_hcore(pos)
density = calc.get_density(pos)
coefficients = calc.get_coefficients(pos)
energies = torch.diag(calc.get_mo_energies(pos))

matrix_grad = get_jacobian(hcore, pos)
print(matrix_grad)

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00, -4.5340e-02],
          [ 0.0000e+00,  0.0000e+00,  4.5340e-02]],

         [[ 0.0000e+00,  0.0000e+00,  4.1503e-03],
          [ 0.0000e+00,  0.0000e+00, -4.1503e-03]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [

In [3]:
from torch.autograd.functional import jacobian


def density_func(pos):
    return calc.get_density(pos)
density_jacobian = jacobian(density_func, pos, strict=True)

density_jacobian

RuntimeError: Output 0 of the user-provided function is independent of input 0. This is not allowed in strict mode.

# Autograd

In [4]:
from dxtb.integrals.wrappers import overlap, hcore
from tqdm import tqdm

par = dxtb.GFN1_XTB

def get_s(pos):
    return calc.integrals.build_overlap(pos)
    # return overlap(numbers, pos, par)

def get_h(pos):
    return hcore(numbers, pos, par)

def get_p(pos):
    return calc.get_density(pos)

def get_e(pos):
    return torch.diag(calc.get_mo_energies(pos))

def get_c(pos):
    return calc.get_coefficients(pos)

max_iter = 10

# for i in tqdm(range(max_iter), desc="overlap libcint"):
#     s_grad = integral.get_gradient(drv_mgr.driver)

for i in tqdm(range(max_iter), desc="hcore"):    
    jacobian = torch.autograd.functional.jacobian(get_h, pos)

# for i in tqdm(range(max_iter), desc="overlap"):    
#     jacobian = torch.autograd.functional.jacobian(get_s, pos, vectorize=False)

# for i in tqdm(range(max_iter), desc="density"):
#     jacobian = torch.autograd.functional.jacobian(get_p, pos, vectorize=False)

# for i in tqdm(range(max_iter), desc="mo energies (diag)"):
#     jacobian = torch.autograd.functional.jacobian(get_e, pos, vectorize=False)

# for i in tqdm(range(max_iter), desc="coefficients"):
#     jacobian = torch.autograd.functional.jacobian(get_c, pos, vectorize=False)


print(jacobian)

hcore: 100%|██████████| 10/10 [00:00<00:00, 11.87it/s]

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00, -4.5340e-02],
          [ 0.0000e+00,  0.0000e+00,  4.5340e-02]],

         [[ 0.0000e+00,  0.0000e+00,  4.1503e-03],
          [ 0.0000e+00,  0.0000e+00, -4.1503e-03]]],


        [[[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00]],

         [[ 0.0000e+00,  0.0000e+00,  0.0000e+00],
          [

# Hcore

In [4]:
from dxtb.integrals.wrappers import overlap, hcore
from dxtb._src.xtb.gfn1 import GFN1Hamiltonian
from dxtb import IndexHelper

# ovlp = overlap(numbers, pos, dxtb.GFN1_XTB) # independenMagnum duck. n the spin (uhf)

par = dxtb.GFN1_XTB
ihelp = IndexHelper.from_numbers(numbers, par)

# h0 = GFN1Hamiltonian(numbers=numbers, positions=pos, par=par, ihelp=ihelp, **dd)

# h = h0.build(pos)


# Overlap

In [6]:
from dxtb.integrals import DriverManager
from dxtb.integrals.factories import new_overlap

from tqdm import tqdm

dd: DD = {"dtype": torch.double, "device": torch.device("cpu")}
ihelp = IndexHelper.from_numbers(numbers, par)
driver_name = 0 # libcint
drv_mgr = DriverManager(driver_name, **dd)
drv_mgr.create_driver(numbers, par, ihelp)
drv_mgr.driver.setup(positions)

integral = new_overlap(drv_mgr.driver_type, **dd)
s = integral.build(drv_mgr.driver)

print(integral.matrix)

s_grad = integral.get_gradient(drv_mgr.driver)

print(f"s shape: {s.shape}")
print(f"s_grad shape: {s_grad.shape}")
print(integral.gradient)

tensor([[ 7.9577e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.4528e-02,
         -1.7739e-02],
        [ 0.0000e+00,  2.3873e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  2.3873e-01,  0.0000e+00,  0.0000e+00,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.3873e-01,  5.0428e-02,
          4.3236e-04],
        [ 4.4528e-02,  0.0000e+00,  0.0000e+00,  5.0428e-02,  7.9577e-02,
          6.7673e-11],
        [-1.7739e-02,  0.0000e+00,  0.0000e+00,  4.3236e-04,  6.7673e-11,
          7.9577e-02]], dtype=torch.float64)
s shape: torch.Size([6, 6])
s_grad shape: torch.Size([6, 6, 3])
tensor([[[-0.0000, -0.0000, -0.0000],
         [ 0.0316, -0.0000, -0.0000],
         [-0.0000,  0.0316, -0.0000],
         [-0.0000, -0.0000,  0.0316],
         [-0.0000, -0.0000,  0.0056],
         [-0.0000, -0.0000, -0.0007]],

        [[-0.0316, -0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000

In [ ]:
import torch

def compute_full_jacobian(overlap_integral, driver):
    """
    Computes the Jacobian of the overlap integral matrix with respect to atomic positions.

    Parameters
    ----------
    overlap_integral : OverlapLibcint
        An instance of the OverlapLibcint class with computed overlap integral.
    driver : IntDriverLibcint
        The integral driver for the calculation.

    Returns
    -------
    jacobian : Tensor
        Jacobian of the overlap integral with respect to atomic positions.
        Shape: (nao, nao, nat, 3)
    """
    # Compute the gradient of the overlap integral matrix
    # Shape: (nao, nao, 3)
    overlap_gradient = overlap_integral.get_gradient(driver)

    # Obtain mapping from basis functions to atoms
    # Shape: (nao,)
    ao_to_atom = driver.drv.ao_to_atom()

    nao = overlap_gradient.shape[0]  # Number of atomic orbitals
    nat = numbers.shape[0]  # Number of atoms

    # Initialize the Jacobian tensor
    jacobian = torch.zeros((nao, nao, nat, 3), dtype=overlap_gradient.dtype, device=overlap_gradient.device)

    # Vectorized implementation to assign gradients to the appropriate atoms
    # Create indices for basis functions
    indices = torch.arange(nao, device=overlap_gradient.device)

    # Create meshgrid of indices
    idx_i, idx_j = torch.meshgrid(indices, indices, indexing='ij')

    # Map basis functions to atoms
    atom_idx_i = ao_to_atom[idx_i]  # Shape: (nao, nao)
    atom_idx_j = ao_to_atom[idx_j]  # Shape: (nao, nao)

    # Expand gradient to match atoms
    grad_expanded = overlap_gradient.unsqueeze(2).expand(-1, -1, 2, 3)  # Shape: (nao, nao, 2, 3)

    # Stack atom indices
    atom_indices = torch.stack((atom_idx_i, atom_idx_j), dim=2)  # Shape: (nao, nao, 2)

    # Flatten tensors for indexing
    flat_i = idx_i.flatten()
    flat_j = idx_j.flatten()
    flat_atoms = atom_indices.reshape(-1, 2)   # Shape: (nao*nao, 2)
    flat_grads = grad_expanded.reshape(-1, 2, 3)  # Shape: (nao*nao, 2, 3)

    # Accumulate gradients per atom
    for k in range(2):
        atom_k = flat_atoms[:, k]
        grad_k = flat_grads[:, k, :]
        jacobian[flat_i, flat_j, atom_k, :] += grad_k

    return jacobian  # Shape: (nao, nao, nat, 3)


jacobian_lbcint = compute_full_jacobian(integral, drv_mgr.driver)

print(jacobian.shape)

def get_s(pos):
    return overlap(numbers, pos, par)


jacobian_autograd = torch.autograd.functional.jacobian(get_s, pos)


print(f"sum of abs diff: {torch.sum(torch.abs(jacobian_lbcint - jacobian_autograd))}")

print(jacobian_lbcint)
print(jacobian_autograd)